In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List
import os

# Data Preprocessing

In [57]:
sp500File = "../Data/Raw Data/ScrapedSP500.csv"
unEmpFile = "../Data/Raw Data/unemployment.csv"
tenYearRateFile = "../Data/Raw Data/10yr_treasury_yield.csv"
recExpFile = "../Data/Raw Data/recession_indicator.csv"
cpiFile = "../Data/Raw Data/consumer_price_index.csv"
vixFile = "../Data/Raw Data/VIX_History.csv"

In [81]:
def createDataFrame(file, year=None):
    #Read data
    df = pd.read_csv(file)
   
    #Capitalize Header
    df = df.rename(columns=str.upper)

    #Create DateTime
    df.insert(0, "DATETIME", pd.to_datetime(df['DATE'])) 

    if year != None:
        #Boolean Mask
        mask = df['DATETIME'].dt.year == year
        df = df[mask]

    #Sanity Check
    if os.path.basename(file) == "ScrapedSP500.csv":
        assert 252 == sum(df[ "DATETIME"].dt.year == year), "Missing Days"
        
    return df 

#Build Full Dataset
def buildFullData(dataSets: pd.DataFrame, listOfDataSets: List, how='left', on='DATETIME') -> pd.DataFrame:
    for data in listOfDataSets:
        dataSets = pd.merge(dataSets, data, how=how, on=on, copy=True, suffixes=(data, dataSets))

    df.apply(lambda columns: val.columns)
    return dataSets

In [82]:
sp500Df = createDataFrame(sp500File, 2021)
unEmpDf = createDataFrame(unEmpFile, 2021)
tenYearRateDf = createDataFrame(tenYearRateFile, 2021)
recExpDf = createDataFrame(recExpFile, 2021)
cpiDf = createDataFrame(cpiFile, 2021)
vixDf = createDataFrame(vixFile, 2021)

In [115]:
economicIndicators = [unEmpDf,
                      tenYearRateDf,
                      recExpDf,
                      cpiDf,
                      vixDf
                      ]

fullData = buildFullData(sp500Df, economicIndicators, how='left', on='DATETIME')

/var/folders/78/bkpkzcyj4j197sb4vtwjnl6r0000gn/T/ipykernel_31008/1311224173.py:25: FutureWarning: Passing 'suffixes' which cause duplicate columns {'DATE_x'} in the result is deprecated and will raise a MergeError in a future version.
  dataSets = pd.merge(dataSets, data, how=how, on=on)
/var/folders/78/bkpkzcyj4j197sb4vtwjnl6r0000gn/T/ipykernel_31008/1311224173.py:25: FutureWarning: Passing 'suffixes' which cause duplicate columns {'DATE_x'} in the result is deprecated and will raise a MergeError in a future version.
  dataSets = pd.merge(dataSets, data, how=how, on=on)


In [110]:
columns = [col for col in res.columns if "DATE_" not in col]
# res.filter(items=columns)  

In [114]:
columns

['DATETIME',
 'HIGH_x',
 'LOW_x',
 'OPEN_x',
 'CLOSE_x',
 'VOLUME',
 'ADJ CLOSE',
 'UNRATE',
 'DGS10',
 'USREC',
 'CPIAUCSL',
 'OPEN_y',
 'HIGH_y',
 'LOW_y',
 'CLOSE_y']

In [113]:
res.shape

(252, 21)